In [2]:
import pandas as pd
import numpy as np
import konlpy
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Okt
import re
import random
import numpy as np

okt = Okt()
komoran=Komoran()
kkma=Kkma()
hannanum=Hannanum()

In [3]:
df = pd.read_excel("/Users/young/git/project/export/비전공자 개발자_2023-01-31.xlsx")
df.rename(columns={'Unnamed: 0':'Rank'},inplace=True)


In [5]:
df.tail(5)

,Rank,title,description,author,timestamp
1045,1045,"문과 개발자, 정말 비전공자도 IT 취업이 가능할까?","문과 개발자, 정말 비전공자도 IT 취업이 가능할까? 문과라서 취업할 길이 보이...",그대 미소에 봄이 돼줄게요,2021.09.18.
1046,1046,"자바 개발자 되는 법, 비전공자에게 적합한 과정은?",자바 개발자 되는 법 비전공자에게 맞는 과정 자바 개발자가 되기 위해 비전공자에...,IT교육및취업전문가 융쌤,2022.06.03.
1047,1047,"소프트웨어개발자되는법, 컴공출신 아닌 비전공자 자바웹포폴...",오늘은 컴공전공생이 아닌 비전공자가 #소프트웨어개발자되는법 #웹개발프로젝트 제작...,월요일에 만든 블로그,2022.01.20.
1048,1048,종로자바학원에서 비전공자도 개발자 취업 가능해요!!,종로자바학원에서 비전공자도 개발자 취업 가능해요!! 비전공자분들의 가장 큰 고민...,따봉아빠의 IT 일상,2017.09.08.
1049,1049,(비전공자 공부순서) 개발자가 하고 싶어요.....,그럼에도 불구하고 디지털노마드를 꿈꾸시거나 개발자의 꿈을 갖고 계신 분 혹은.....,초보개발자 _ 왕곰,2022.09.13.


In [ ]:
-----WIPWIPWIPWIPWIPWIPWIPWIPWIP------
def word_parser_wip(df):
    structure_list = []
    element_list = []

    for idx,rows in df.iterrows():
        structure = okt.pos(rows[1])
        element = okt.morphs(rows[1])
        
        structure_list.append([i[1] for i in structure])
        element_list.append([i for i in element])

    return structure_list, element_list
structure_list, element_list = word_parser(df)

structure_list

In [ ]:
-----WIPWIPWIPWIPWIPWIPWIPWIPWIP------
def word_parser(df):
    structure_list = []
    element_list = []

    for idx,rows in df.iterrows():
        split_phrases = okt.phrases(sentence)
        split_morphs = okt.morphs(sentence, stem=False)

    #명사 중복 제거 후 구조화
    tmp_structure = []
        for idx,tup in enumerate(okt.pos(sentence)):
            if idx > 0 and tmp == 'Noun' and tup[1] == 'Noun':
                continue

        tmp = okt.pos(sentence)[idx][1]
        tmp_structure.append(tup[1])

    print(tmp_structure)

    #adjective 분할기
    adjective_singular = []
    adjective_plural = []

    for idx,tup in enumerate(okt.pos(sentence,join=True)):
        pos = re.search(('(?<=\/)[a-zA-Z]*'),tup)[0]
        if pos == 'Adjective':
            # print(pos)
            adjective = re.search(('.+(?=\/)'),tup)[0]
            print (adjective)



# print(b)
# print(okt.pos(sentence,join=True))
# print(hannanum.pos(sentence))
# print(okt.pos(a[4]))
# print(okt.pos(sentence))

In [6]:
df

,Rank,title,description,author,timestamp
0,0,비전공자 개발자 : 준비하기 적합한 케이스 및 취업률 높이기,"직업, 개발자를 목표로 삼는 경우가 많습니다. 하지만, 비전공자 개발자 취업이 ...",주들의 그리 대단하진 않지만 굉장한 이야기,어제
1,1,코딩 개발자 취업 비전공자의 가능성과 연봉 알아보기,등등 비전공자 개발자로 취업을 준비할 때 가장 중요한 건 직무 선택입니다. 코딩...,인공지능 특이점이 찾아올거 같아서 만든 블로그,2023.01.02.
2,2,코딩 개발자 비전공자 취업 현실,비전공자 코딩 개발자 취업 현실 - 초봉 2600만 원 - 파견직 개발자 단기간...,인테리어,5시간 전
3,3,"자바개발자 : 비전공자, 취업성공의 지름길","자바개발자 : 비전공자, 취업성공의 지름길 IT개발자 중 가장 많은 수요를 기록...",비비디 바비디 부,2022.12.07.
4,4,비전공자개발자 학원에서 취업성공한 후기,"있는 비전공자분들 비전공자개발자 취업을 준비한다면, 이미 비슷한 조건과 상황.....",할일은 없지만 놀기만하지않ㅇ,2022.12.08.
...,...,...,...,...,...
1045,1045,"문과 개발자, 정말 비전공자도 IT 취업이 가능할까?","문과 개발자, 정말 비전공자도 IT 취업이 가능할까? 문과라서 취업할 길이 보이...",그대 미소에 봄이 돼줄게요,2021.09.18.
1046,1046,"자바 개발자 되는 법, 비전공자에게 적합한 과정은?",자바 개발자 되는 법 비전공자에게 맞는 과정 자바 개발자가 되기 위해 비전공자에...,IT교육및취업전문가 융쌤,2022.06.03.
1047,1047,"소프트웨어개발자되는법, 컴공출신 아닌 비전공자 자바웹포폴...",오늘은 컴공전공생이 아닌 비전공자가 #소프트웨어개발자되는법 #웹개발프로젝트 제작...,월요일에 만든 블로그,2022.01.20.
1048,1048,종로자바학원에서 비전공자도 개발자 취업 가능해요!!,종로자바학원에서 비전공자도 개발자 취업 가능해요!! 비전공자분들의 가장 큰 고민...,따봉아빠의 IT 일상,2017.09.08.


In [7]:
df['structure'] = ''


In [8]:
def pos_seperator(pos):
    #품사
    pos = re.search(('(?<=\/)[a-zA-Z]*'),tup)[0]
    #형태소
    morphs = re.search(('.+(?=\/)'),tup)[0]

def add_scores():
    return 

def word_parser(df):
    morphs_list = []
    phrase_list = []
    structure_list = []
    # count_morphs_list = []
    # count_phrase_list = []
    # df 반복문 돌며 어절, 형태소 분해, 문장구조 분해 
    for idx,rows in df.iterrows():
        split_phrases = okt.phrases(rows[1])
        split_morphs = okt.pos(rows[1], join=True)

        for i in split_phrases:
            phrase_list.append(i + '/' + 'Noun')
        for i in split_morphs:
            if 'Punctuation' in i:
                continue
            morphs_list.append(i)
    #명사 중복 제거 후 구조화
        # tmp_structure = []
        tmp_structure = ''
        for jdx,tup in enumerate(okt.pos(rows[1])):
            if jdx > 0 and tmp == 'Noun' and tup[1] == 'Noun':
                continue
            tmp = okt.pos(rows[1])[jdx][1]
            # structure_list.append(tup[1])
            # tmp_structure += '_' + tup[1] if jdx != 0 or jdx != len(okt.pos(rows[1])) - 1 else tup[1]
            # if jdx != 0:
            #     tmp_structure += '_' + tup[1]
            # else:
            #     tmp_structure += tup[1]
            tmp_structure += '_' + tup[1]
            # structure_list.append([i for i in tup[1]])
        structure_list.append(tmp_structure)

    #unique값 찾기
    morphs_set = list(set(morphs_list))
    phrase_set = list(set(phrase_list))
    structure_set = list(set(structure_list))
    #빈도 카운트
    print(len(phrase_set))
    for i in range(len(morphs_set)):
        #두 리스트간 중복 삭제
        if morphs_set[i] in phrase_set:
            phrase_set.remove(morphs_set[i])
        morphs_set[i] = morphs_set[i] + '/' + str(morphs_list.count(morphs_set[i]))

    for i in range(len(phrase_set)):
        phrase_set[i] = phrase_set[i] + '_joint/' + str(phrase_list.count(phrase_set[i]))

    for i in range(len(structure_set)):
        structure_set[i] = structure_set[i] + '/' + str(structure_list.count(structure_set[i]))

    return phrase_set,morphs_set,structure_set,structure_list

phrase_set,morphs_set,structure_set,structure_list = word_parser(df)



3251


In [9]:
df_orgin_structure = []
for i in structure_list:
    
    split = re.split(('_'),i)
    del split[0]

    df_orgin_structure.append(split)

df['structure'] = df_orgin_structure



In [18]:
#단어 토큰 df화
frame = {
    'word' : [],
    'pos' : [],
    'cnt' : []
}
word_st = phrase_set + morphs_set
for i in word_st:
    try:
        split = re.split(('/'),i)
        # print(split[1],split[2],split[3])
        frame['word'].append(split[0])
        frame['pos'].append(split[1])
        frame['cnt'].append(int(split[2]))
    except:
        # print(i)
        continue

frame
df2 = pd.DataFrame(frame)
df2 = df2.sort_values(by="cnt",ascending=False)
df2

,word,pos,cnt
2994,비,Noun,1063
3524,전공자,Noun,1062
3118,개발자,Noun,1029
2029,비전공자,Noun_joint,659
3053,취업,Noun,599
...,...,...,...
1433,이력서 작성법,Noun_joint,1
1434,PDF,Noun_joint,1
1435,기반 스마트웹,Noun_joint,1
1436,국비지원학원,Noun_joint,1


In [11]:
# 문장구조 df화
structure_frame = {
    'structure' : [],
    'cnt' : []
}
for i in structure_set:
    split_cnt = re.split(('/'),i)
    cnt = int(split_cnt[-1])
    
    split = re.split(('_'),i)
    del split[0]
    split[-1] = re.split(('/'),split[-1])[0]
    # print(cnt)
    structure_frame['structure'].append(split)
    structure_frame['cnt'].append(cnt)
df3 = pd.DataFrame(structure_frame)
df3 = df3.sort_values(by="cnt",ascending=False)
df3

,structure,cnt
379,[Noun],40
669,"[Noun, Punctuation, Noun]",19
241,"[Noun, Josa, Noun]",18
498,"[Noun, Punctuation]",11
770,"[Noun, Punctuation, Noun, Punctuation]",9
...,...,...
277,"[Punctuation, Noun, Punctuation, Noun, Josa, N...",1
280,"[Noun, Josa, Punctuation, Noun]",1
281,"[Noun, Alpha, Punctuation, Alpha, Punctuation,...",1
283,"[Noun, Punctuation, Noun, Verb, Verb, Noun, Pu...",1


In [19]:
df3_pop = df3[df3['cnt'] > 1]

for i in range(1):
    sample_structure = random.choices(list(df3_pop['structure']),weights=[i for i in df3_pop['cnt']])[0]



In [20]:
sample_structure_dict = dict.fromkeys(sample_structure)

In [21]:
df2

,word,pos,cnt
2994,비,Noun,1063
3524,전공자,Noun,1062
3118,개발자,Noun,1029
2029,비전공자,Noun_joint,659
3053,취업,Noun,599
...,...,...,...
1433,이력서 작성법,Noun_joint,1
1434,PDF,Noun_joint,1
1435,기반 스마트웹,Noun_joint,1
1436,국비지원학원,Noun_joint,1


In [22]:
#문장구조와 단어 조합
def random_choice(sample_list, weight):
    sample = random.choices(sample_list,weights=weight)
    return sample[0]

for pos in sample_structure:
    if pos == 'Noun':
        coin_flip = random_choice(["Noun","Noun_joint"],[0.2,0.8])
        df2_n = df2[df2['pos'] == coin_flip]
    else:
        df2_n = df2[df2['pos'] == pos]
    df2_n.reset_index(drop=False)
    df2_n.sort_values(by='cnt',ascending=False)

    match_senetence = []
    for i,row in df.iterrows():
        if sample_structure == row[5]:
            match_senetence.append(row[1])

    words = []
    
    for i in range(100):
        if pos == 'Punctuation':
            sample = random_choice(["?","!","??","!!"],[1,1,1,1])

        else:
            sample = random_choice(list(df2_n['word']),[i for i in df2_n['cnt']])

        words.append(sample)
    sample_structure_dict[pos] = words

print(sample_structure)
print(match_senetence)
print(sample_structure_dict)

['Noun', 'Modifier', 'Noun', 'Punctuation']
['개발자취업연계 비전공자 백엔드 과정!', '비전공자 자바개발자 취업학원 연봉올리기!!']
{'Noun': ['생활 코딩 인강', '2022 이기적 SQL 개발자 비전공자 독학 공부 인강', '비전공자 개발자', '빅데이터개발자 비전공자', '비전공자 취업 대비 및', '비전공자', '비전공자', '학사학위와 직무 능력', '공부 과정', '100', '비전공자 개발자들', '2022 이기적 SQL 개발자 비전공자', '측면과 자격증', '46회 SQL개발자', '비전공자 개발자', '빅데이터개발자 비전공자 취업', '간 문과 여자', '비전공자', '앱개발자', '부산개발자취업 학원', '자체 개발자', '취업 준비방법', '필요 소양 및 비전공자 취업', '개발자된 후기', '비전공자 개발자 전직 준비', '청주컴퓨터학원 전액무료', 'step', '문과 졸업', '비전공자 개발자 취업공부', '2022 이기적 SQL', '부산백엔드학원', '비전공자 도 가능하다는 말 은', '비전공자', '46회', 'IT', '민코딩', '비전공자', '30', '비전공자 빅데이터', 'Case', '빅데이터 개발자 비전공자', '비전공자 웹개발자 취업 성공', '국비지원 프로그래밍', '코딩개발자', 'IT개발자 인력 전쟁', '중도 포기율', 'SQLD', '비전공자', '비전공자', '크래프톤 정글 SW', '비전공자 개발자', '엘리스 코딩 AI트랙 5기 합격후기', '비전공자', '개발자 자격증', '비전공자', '강사 인터뷰', '인력 부족', '비전공자개발자', '성공한 비전공자', '코딩개발자 취업 준비', '비개발자', '빅데이터 개발자 되는법', '비전공자', '비전공자 IT취업', '종로코딩학원', '비전공자', '비전공자 개발자', 'EP', 'IT개발자', '점핏 비전공자 개발자 취업', '가능한 개발자 취업', 'JAVA', '자바웹개발자 비전공자 취업 준비 전문 